In [ ]:
class PushdownAutomaton0():
    

    def initialize(self, q, sigma, gamma, delta, q0, f, empty_symbol):        

        self.q = q           # list of strings representing automaton states
        self.sigma = sigma   #list of symbols containing alphabet
        self.gamma = gamma   #list of symbols containing the stack alphabet
        self.delta = delta   # Dictionary of pairs and a list of possible states
        self.q0 = q0         # Initial state
        self.empty_symbol = empty_symbol # epsilon or any symbol used
        self.f = f           # List of final states
        self.current = self.q0  # Current state of the Automaton 
        self.stack = [] #keeps track of stack
        
        
    def load_from_file(self, path):
        """
        Loads pushdown from file provided by professor
        """
        import re  
        
        # Read file
        with open(path) as f:
            # Store in a list opf lines
            file_lines = f.readlines()

        
        # clean line by line
        for i in range(len(file_lines)):
            file_lines[i] = re.sub("\{|\ |\}|}","",file_lines[i])#remove characters
            file_lines[i] = file_lines[i].replace('\t',"")#remove tabs
            file_lines[i] = file_lines[i].replace('\n',"")#remove line breaks
        
        # Store iinitial state
        q0 = file_lines[4]

        #Constructing lists
        q = file_lines[0].split(',')
        sigma = file_lines[1].split(',')
        gamma = file_lines[2].split(',')
        f = file_lines[-1].split(',')
        
        # Creating delta dictionary

        first_delta = file_lines[3].replace('}','').replace('{','').replace(' ','').split(',')
        new_delta = {}
        for a in first_delta:    
            # store empty symbol
            if 'epsilon' in a:
                empty_symbol = 'epsilon'#done this way because examples provided use epsilon, if user wants a different symbol this can be done with initialize method
            elif 'Epsilon' in a:
                empty_symbol = 'Epsilon'#done this way because examples provided use Epsilon, if user wants a different symbol this can be done with initialize method
            # key is before second "-", value is after
            index = [i for i, n in enumerate(a) if n == '-'][1]#finds second -
            key = a[0:index]
            myvalue = a[index+1:]
            new_delta[key] = myvalue
        #this makes delta dictionary elements look like this: {original state - value - stack cost: next state - stack adding}
        #set parameters using initialize function
        self.initialize(q, sigma, gamma, new_delta, q0, f, empty_symbol)
    
    #will create empty_transitions function before process_symbol since the previous will be used in the further
    def one_empty_trans(self, state):
        """
        recursive function that will be used in empty_transitions
        """
        # Create a list of visited states
        visited = []
        # Create a list of possible states
        possible_states = [state]
        # Iterate until all possible states have been visited
        while len(possible_states) > 0:
            # Pop the last state from the list of possible states
            current_state = possible_states.pop()
            # Add the current state to the list of visited states
            visited.append(current_state)
            # Create an empty transition from the current state
            empty_trans = current_state+'-'+self.empty_symbol
            # List of possible next states
            possibles = []
            for key in self.delta.keys():
                if empty_trans in key: 
                    myvalue = self.delta[key]
                    divider = myvalue.index('-')
                    mystate = myvalue[divider+1:divider+3]
                    mycost = myvalue[:divider]#cost in stack
                    if mycost == self.empty_symbol:#if cost is none, state is reachable with no changes to stack
                        possibles.append(mystate)
                    elif len(self.stack) == 0:#if cost is NOT none, and stack is empty, state is not reachable
                        continue
                    elif mycost == self.stack[-1]:#if cost is the same as stack´s last element, state is reachable
                        possibles.append(mystate)
                        print(f"NOTE: transition to {mystate} is possible but would cost last stack element")
                    else:#if cost is NOT the same as stack´s last element, state is unreachable
                        continue
            # Iterate over the list of possible next states
            for p_state in possibles:
                # If the possible state has not been visited or added to the list of possible states, add it to the list
                if p_state not in visited and p_state not in possible_states:
                    possible_states.append(p_state)
        # Return the list of visited states
        return visited


    def empty_transitions(self, state):
        """
        returns a list with the reachable states with empty transitions from given state
        """
        # Call the one_empty_trans function to get the list of reachable states
        final_states = self.one_empty_trans(state)
        # If there are no reachable states, return a list containing only the current state
        if len(final_states) == 0:
            return [state]
        # Create a list to store the final result
        result = []
        # Iterate over the list of reachable states
        for element in final_states:
            # If the element is not already in the result list, add it
            if element not in result:
                result.append(element)
        # Return the final list of reachable states
        return result

    def process_symbol(self, state, symbol):
        """
        Processes one symbol in pushdown and returns possible next states in a list
        """
        pos_next_states=[]
        print(self.stack)
        last_stack=len(self.stack)-1
        #If the symbol asks for a key, review of the stack is need, else not
        for k,v in self.delta.items():
            if(state==k.split("-")[0] and symbol==k.split("-")[1]):
                if(v.split("-")[0]=="epsilon"):
                    pos_next_states.append(v.split("-")[1])
                elif(v.split("-")[0]!="epsilon"):
                        print("It is needed this in the stack: ", v.split("-")[0])
                        if(v.split("-")==last_stack):
                            print("You have it and is a possible state ")
                            pos_next_states.append(v.split("-")[1])
                        else:
                            pos_next_states.append(v.split("-")[1])
                
        return pos_next_states

    def process_word(self, word):
        """
        Processes a word and returns if accepted or not by a boolean
        Warning: Processes words to up to 2 symbols
        """
        last_stack=len(self.stack)-1
        switch=0

        for symbol in word:
            self.current="q2"
            #Each symbol of the word calls process_symbol
            possibles_st = self.process_symbol(self.current, symbol)
            print("symbol: ",symbol)
            push=""
            pop=""
            for k,v in self.delta.items():
                removed=""
                #Initial condition for location of the transition
                if(k.split("-")[0]==self.current and k.split("-")[1]==symbol and v.split("-")[1] in possibles_st):
                    #Different cases for comparison of the symbols and the transition.
                    if(v.split("-")[0]=="epsilon" and v.split("-")[2]=="epsilon"):
                        self.current=v.split("-")[1]
                        pass
                    elif(v.split("-")[0]=="epsilon" and v.split("-")[2]!="epsilon"):
                        self.current=v.split("-")[1]
                        self.stack.append(v.split("-")[2])
                        push=self.stack[last_stack]
                    elif(v.split("-")[0]!="epsilon" and v.split("-")[2]=="epsilon" and self.stack[last_stack]==v.split("-")[0]):
                        self.current=v.split("-")[1]
                        pop=self.stack[last_stack]
                        self.stack.pop()
                    elif(v.split("-")[0]!="epsilon" and v.split("-")[2]!="epsilon"):
                        self.current=v.split("-")[1]
                        pop=self.stack[last_stack]
                        self.stack.pop()
                        self.stack.append(v.split("-")[2])
                        push=self.stack[last_stack]
                    else:
                        print("No movement allowed")
                        switch=1
                        break
                    print("This key was pop: ",pop)
                    print("This key was push: ",push)
                    #It makes push or pop depending on what was needed. 
                
        r=possibles_st[0]#the possible state in case of being close to the final state.

        print("posible:",r)
        #If the next state is final state or the current state is final state.
        if(switch==0 and self.current in self.f or "q4" in self.process_symbol(r, "epsilon")):
            return True
        else:
            return False

    def __str__(self):
        '''
        Prints the current automaton configuration
        '''
        ToPrint = ("CURRENT AUTOMATON CONFIGURATION\n"+"\nInternal States ="+ str(self.q) +"\nAlphabet =" +str(self.sigma) +
    "\nStack Alphabet =" +str(self.gamma) +"\nPossible states =" +str(self.delta) +"\nInitial State =" +str(self.q0) +"\nEmpty transition =" +str(self.empty_symbol) +"\nFinal States="+str(self.f)+
    "\n\nThe Current State of the automaton is = "+str(self.current))
        return print(ToPrint)  


    def valid(self):
        """
        checks if pushdown automaton is valid and returns a boolead
        """
        # Verify that there is only a initial element
        List = []
        List.append(self.q0)
        
        rules = [len(List) == 1,
                # F is a subset of q 
                all(item in self.q for item in self.f),
                # q0 belong the set of states (q)
                self.q0 in self.q]
        
        # Count the states
        count = 0
        switch_delta=0
        # Count the extra states that are in a state with same trasition for check the symbols
        check_symbol = []
        # Check empty symbol
        check_empty_symbol =''
        
        # Check the previous rules for check others rules
        if all(rules):
            # Iterate over delta
            for k, v in self.delta.items():
                # Check every transitions are from and to states of q
                if (k.split("-")[0] in self.q and v.split("-")[1] in self.q):
                    count += 1
                    # If the transition is different of epsilon
                    if (k.split('-')[1] != 'epsilon'):
                        check_symbol.append(k.split('-')[1])
                    # If the transition is epsilon
                    else:
                        check_empty_symbol = k.split('-')[1]
                if (v.split("-")[0] in self.gamma or v.split("-")[0]=='epsilon' and v.split("-")[2] in self.gamma or v.split("-")[2]=='epsilon'):
                    switch_delta=switch_delta+0
                else:
                    switch_delta=1
        
        # Check all transitions are from and to states of q and 
        # all symbols belong to sigma and that the symbol belongs to the empty symbol 
        if (count == (len(self.delta)) and all(item in self.sigma for item in check_symbol) and (check_empty_symbol == self.empty_symbol) and (switch_delta==0)):
            return True 
        # If the automaton don't comply with any rule 
        else:
            raise Exception('NotValidConfigurationError')
        return None

## Testing functions with file provided by professor

In [ ]:
#Testing valid function
pru = PushdownAutomaton0()
pru.load_from_file("notvalid_examplepushdown_automata_01.txt")
pru.valid()

Exception: NotValidConfigurationError

In [ ]:
#Correct file
p = PushdownAutomaton0()
p.load_from_file("pushdown_automata_01.txt") 
p.valid()

True

In [ ]:
p.__str__()

CURRENT AUTOMATON CONFIGURATION

Internal States =['q1', 'q2', 'q3', 'q4']
Alphabet =['0', '1']
Stack Alphabet =['0', '$']
Possible states ={'q1-epsilon': 'epsilon-q2-$', 'q2-0': 'epsilon-q2-0', 'q2-1': '0-q3-epsilon', 'q3-1': '0-q3-epsilon', 'q3-epsilon': '$-q4-epsilon'}
Initial State =q1
Empty transition =epsilon
Final States=['q1', 'q4']

The Current State of the automaton is = q1


In [ ]:
#Testing empty_transitions
p.empty_transitions("q3")

['q3']

In [ ]:
p.empty_symbol

'epsilon'

In [ ]:
p.delta
#dedonde--que escribo: que tomo--adondellego--que metp.pro

{'q1-epsilon': 'epsilon-q2-$',
 'q2-0': 'epsilon-q2-0',
 'q2-1': '0-q3-epsilon',
 'q3-1': '0-q3-epsilon',
 'q3-epsilon': '$-q4-epsilon'}

In [ ]:
print("q2 with a 0:", p.process_symbol("q2","0"))
print("q3 with a 1",p.process_symbol("q3","1"))
print("q1 with epsilon",p.process_symbol("q1","epsilon"))
print("q3 with epsilon ",p.process_symbol("q3",'epsilon'))

[]
q2 with a 0: ['q2']
[]
It is needed this in the stack:  0
q3 with a 1 ['q3']
[]
q1 with epsilon ['q2']
[]
It is needed this in the stack:  $
q3 with epsilon  ['q4']


In [ ]:
p.process_word("00")

[]
symbol:  0
This key was pop:  
This key was push:  0
['0']
symbol:  0
This key was pop:  
This key was push:  0
posible: q2
['0', '0']


False

In [ ]:
p.process_word("01")

['0', '0']
symbol:  0
This key was pop:  
This key was push:  0
['0', '0', '0']
It is needed this in the stack:  0
symbol:  1
This key was pop:  0
This key was push:  
This key was pop:  0
This key was push:  
posible: q3
['0']
It is needed this in the stack:  $


True

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=955b83f2-8744-420b-a8b8-dae693fbb1e6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>